# Network Analysis
This file plots the feeders and finds the furthest loads from the transformers

In [ ]:
from pathlib import Path

!pip install networkx
!pip install xlrd
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches

import numpy as np
import seaborn as sns
sns.set_style('whitegrid')


In [ ]:
base_path = Path.cwd()
print(f"{base_path=}")
lv_models_path = base_path.joinpath("NandoLV").joinpath("LVNetworkModels")
print(f"{lv_models_path=}")

In [ ]:
def construct_network(feeder_data):
    # Create a directed graph using NetworkX
    G = nx.Graph()

    # Add edges to the graph based on the feeder data
    for col, row in feeder_data.iterrows():
        source_node = row['NodeA']
        end_node = row['NodeB']
        distance = row['D[m]']
        phases = row['Phase']
        load = row['Load']

        G.add_edge(source_node, end_node, distance=distance, phases=phases, load=load)

    return G

def plot_network(network, feeder_data, ax):
    # Get X and Y coordinates for each node
    node_positions = {row['Node']: (row['X'], row['Y']) for _, row in xy_position.iterrows()}

    # Create a colormap based on the number of phases
    edge_colors = ['blue', 'green', 'red', 'black'] 

    # Map edge colors based on the phases
    edge_color_mapping = {i: edge_colors[i - 1] for i in range(1,5)}

    # Extract phase information for each edge
    edge_phases = [network.edges[edge]['phases'] for edge in network.edges]
    ## accounting for load 1 not having an entry
    node_colors = ["skyblue"] + ['orange' if feeder_data.loc[node]['Load'] == 1 else 'none' for node in range(len(feeder_data))]
    node_size = [5] + [1]*len(feeder_data)


    # Plot the network using matplotlib with specified node and edge colors
    nx.draw(network, ax = ax, pos=node_positions, with_labels=False, node_size=node_size, node_color=node_colors, font_size=8, edge_color=[edge_color_mapping[phase] for phase in edge_phases], width = 0.5)
    
    # Label nodes where loads are at NodeB
    labeled_nodes = set()
    overlapping_nodes = set()
    seen_coordinates = set()

    for node, (x, y) in node_positions.items():
        load_data = feeder_data[feeder_data['NodeB'] == node]['Load']
        if not load_data.empty and load_data.values[0] == 1:
            labeled_nodes.add(node)
            #plt.text(x, y, f"{node:.0f}", fontsize=8, ha='right', va='bottom')
            if (x, y) in seen_coordinates:
                overlapping_nodes.add(node)
            else:
                seen_coordinates.add((x, y))
    
    # Print the number of labeled nodes
    num_labeled_nodes = len(labeled_nodes)
    if num_labeled_nodes > 0:
        print(f"There are {num_labeled_nodes} labeled nodes with Load=1.")
        print(f"Labeled nodes: {labeled_nodes}")

        # Print the number of overlapping labeled nodes
        num_overlapping_nodes = len(overlapping_nodes)
        if num_overlapping_nodes > 0:
            print(f"{num_overlapping_nodes} of them are overlapping.")
            print(f"Overlapping nodes: {overlapping_nodes}")
        else:
            print("There are no overlapping labeled nodes.")
    else:
        print("There are no labeled nodes with Load=1.")
    # Create a custom legend
    legend_patches = [mpatches.Patch(color=edge_colors[i], label=f'Phase {i+1}') for i in range(3)] + [mpatches.Patch(color=edge_colors[3], label=f'3 Phase')]
    leg = ax.legend(handles=legend_patches, title='Phases', fontsize = 8)
    leg.get_title().set_fontsize('8')

In [ ]:
def OpenFile(Net,Feed):
    try:
        feeder_path = lv_models_path.joinpath(f"Network_{Net}").joinpath(f"Feeder_{Feed}")
        with open(feeder_path.joinpath("Feeder_Data.xls"), 'rb') as f:
            feeder_data = pd.read_excel(f)
        with open(feeder_path.joinpath("XY_Position.xls"), 'rb') as f:
            xy_position = pd.read_excel(f)
        return True, feeder_data, xy_position
    except:
        return False, 0, 0

In [ ]:
def count_overlapping_nodes(feeder_data, xy_position):
    # Label nodes where loads are at NodeB
    labeled_nodes = set()
    overlapping_nodes = set()
    seen_coordinates = set()
    
    node_positions = {row['Node']: (row['X'], row['Y']) for _, row in xy_position.iterrows()}

    for node, (x, y) in node_positions.items():
        load_data = feeder_data[feeder_data['NodeB'] == node]['Load']
        if not load_data.empty and load_data.values[0] == 1:
            labeled_nodes.add(node)
            if (x, y) in seen_coordinates:
                overlapping_nodes.add(node)
            else:
                seen_coordinates.add((x, y))

    # Print the number of labeled nodes
    num_labeled_nodes = len(labeled_nodes)
    if num_labeled_nodes > 0:
        print(f"There are {num_labeled_nodes} labeled nodes with Load=1.")
        #print(f"Labeled nodes: {labeled_nodes}")

        # Print the number of overlapping labeled nodes
        num_overlapping_nodes = len(overlapping_nodes)
        if num_overlapping_nodes > 0:
            print(f"{num_overlapping_nodes} of them are overlapping.")
            #print(f"Overlapping nodes: {overlapping_nodes}")
        else:
            print("There are no overlapping labeled nodes.")
    else:
        print("There are no labeled nodes with Load=1.")
    return num_labeled_nodes, num_overlapping_nodes


In [ ]:

distances = []
nodes = []
feed_info = {}
number_cust = []
for i in range(1,27):
    feed = 0
    while True:
        feed +=1
        exception, feeder_data, xy_position = OpenFile(i, feed)
        if not exception:
            break
        network = construct_network(feeder_data)
        node_positions = {node: (row['X'], row['Y']) for node, row in xy_position.iterrows()}
        
        shortest_distances = {}
        Load_Number = 1
        # NODE B HAS LOAD
        for load_node in feeder_data[feeder_data['Load'] == 1]['NodeB']:
            shortest_distances[Load_Number] = nx.shortest_path_length(network, source=load_node, target=1, weight="distance")
            Load_Number +=1
        

        
        
        #print(f"Network {i}, Feeder {feed}")
        number_cust.append(Load_Number)
        #feed_info[f"{i}, {feed}"] = count_overlapping_nodes(feeder_data, xy_position)
        
         
        max_pair = max(shortest_distances.items(), key=lambda x: x[1])
        #print(f"node {max_pair[0]} has length {max_pair[1]}m")
        distances.append(max_pair[1])
        #nodes.append(max_pair[0])



print("Number Customers:", max(number_cust), min(number_cust))
print("Max Line Lengths:", max(distances),min(distances))

In [ ]:
"""
Load_Number = 1
shortest_distances = {}
for load_node in feeder_data[feeder_data['Load'] == 1]['NodeB']:
    shortest_distances[Load_Number] = nx.shortest_path_length(network, source=load_node, target=1, weight="distance")
    Load_Number +=1
print("the nodes in order of distance are; ", np.array(sorted(shortest_distances.items(), key=lambda x: x[1]))[:,0])
#print(f"furtest node from transformer has distance {max(shortest_distances)}")
max_pair = max(shortest_distances.items(), key=lambda x: x[1])
print(f"node {max_pair[0]} has length {max_pair[1]}m")
"""

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (6.3, 3))

_, feeder_data, xy_position =OpenFile(1,1)
network = construct_network(feeder_data)
plot_network(network, feeder_data, ax[0])

_, feeder_data, xy_position =OpenFile(2,1)
network = construct_network(feeder_data)
plot_network(network, feeder_data, ax[1])
plt.rcParams['font.family'] = 'Times New Roman'


ax[0].set_title("Network 1 Feeder 1", fontsize = 8)
ax[1].set_title("Network 2 Feeder 1", fontsize = 8)
plt.tight_layout()
plt.savefig(base_path.joinpath("Images").joinpath("example_network.svg"))

In [ ]:
distances

In [ ]:
len(nodes)

In [ ]:
print(nodes)